## Imports  
---

In [1]:
# Import the required libraries and dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.style.use('ggplot')
%matplotlib inline

## Load Data  
---

In [2]:
# Import the data by reading in the CSV file and setting the DatetimeIndex 
# Review the first 5 rows of the DataFrame
fund_df = pd.read_csv(
    Path('Resources/whale_navs.csv'),
    index_col='date',
    parse_dates=True,
    infer_datetime_format=True
)
fund_df.columns = [name.lower().replace(' ', '_') for name in fund_df.columns]
fund_df.head()

,soros_fund_management_llc,paulson_&_co.inc.,tiger_global_management_llc,berkshire_hathaway_inc,s&p_500
date,,,,,
2014-10-01,31.950240,14.991826,59.977830,51.948712,194.35
2014-10-02,31.936110,14.994072,59.978626,51.957619,194.38
2014-10-03,31.969707,14.999596,60.002264,52.022484,196.52
2014-10-06,32.048215,14.999471,60.006244,52.036387,196.29
2014-10-07,31.964216,14.994720,59.993735,52.005864,193.26


## Sharpe Ratio  
---

In [18]:
# Calculate daily returns.
daily_returns = fund_df.pct_change().dropna(axis=0, how='all').copy()

# Hold out the last year of returns to test
# selected portfolios againgst.
daily_returns_analyze = daily_returns[:-252]
daily_returns_holdout = daily_returns[-252:]

daily_returns_analyze.head()

,soros_fund_management_llc,paulson_&_co.inc.,tiger_global_management_llc,berkshire_hathaway_inc,s&p_500
date,,,,,
2014-10-02,-0.000442,0.000150,0.000013,0.000171,0.000154
2014-10-03,0.001052,0.000368,0.000394,0.001248,0.011009
2014-10-06,0.002456,-0.000008,0.000066,0.000267,-0.001170
2014-10-07,-0.002621,-0.000317,-0.000208,-0.000587,-0.015436
2014-10-08,0.000390,0.000369,0.000242,0.000842,0.017489


In [19]:
# Calculate the annual average return data for the for fund portfolios and the S&P 500
# Use 252 as the number of trading days in the year
# Review the annual average returns sorted from lowest to highest
avg_annl_rtn = (daily_returns_analyze.mean() * 252)

# Calculate the annualized Sharpe Ratios for each of the 4 portfolios and the S&P 500.
# Review the Sharpe ratios sorted lowest to highest

# Calculate the annualized standard deviaton.
std_dev_annl = (daily_returns_analyze.std() * np.sqrt(252))

# Calculate the Sharpe Ratio.
risk_ratios_df = pd.DataFrame(
    avg_annl_rtn / std_dev_annl,
    columns=['sharpe_ratio']
)
risk_ratios_df

,sharpe_ratio
soros_fund_management_llc,-0.218074
paulson_&_co.inc.,-0.095913
tiger_global_management_llc,0.564186
berkshire_hathaway_inc,0.528679
s&p_500,0.714305


## Sortino Ratio

In [20]:
# Calculate the annualized standard deviation
# for losses only.
neg_std_dev_annl = (daily_returns_analyze[daily_returns_analyze < 0].std() * np.sqrt(252))

# Calculate the Sortino ratio
risk_ratios_df['sortino_ratio'] = avg_annl_rtn / neg_std_dev_annl
risk_ratios_df

,sharpe_ratio,sortino_ratio
soros_fund_management_llc,-0.218074,-0.246034
paulson_&_co.inc.,-0.095913,-0.117876
tiger_global_management_llc,0.564186,0.666718
berkshire_hathaway_inc,0.528679,0.701791
s&p_500,0.714305,0.894260


## Calmar Ratio  
---  

In [31]:
# Calculate the max drawdown for each portfolio.
cum_returns = (daily_returns_analyze + 1).cumprod()
peak = cum_returns.expanding(min_periods=1).max()
drawdown = ((cum_returns / peak) - 1).min()
drawdown

soros_fund_management_llc     -0.064800
paulson_&_co.inc.             -0.063287
tiger_global_management_llc   -0.017340
berkshire_hathaway_inc        -0.120894
s&p_500                       -0.201785
dtype: float64

In [32]:
# Calculate the Calmar Ratio
risk_ratios_df['calmar_ratio'] = avg_annl_rtn / abs(drawdown)
risk_ratios_df

,sharpe_ratio,sortino_ratio,calmar_ratio
soros_fund_management_llc,-0.218074,-0.246034,-0.056719
paulson_&_co.inc.,-0.095913,-0.117876,-0.038183
tiger_global_management_llc,0.564186,0.666718,0.274038
berkshire_hathaway_inc,0.528679,0.701791,0.209938
s&p_500,0.714305,0.894260,0.486772


## Information Ratio  
---

In [38]:
# Calculate the Information Ratio
returns_difference = daily_returns_analyze.drop('s&p_500', axis=1).sub(
    daily_returns['s&p_500'],
    axis='index'
)
annl_volatility = returns_difference.std() * np.sqrt(252)
risk_ratios_df['information_ratio'] = (returns_difference.mean() * 252) / annl_volatility
risk_ratios_df

,sharpe_ratio,sortino_ratio,calmar_ratio,information_ratio
soros_fund_management_llc,-0.218074,-0.246034,-0.056719,-0.792278
paulson_&_co.inc.,-0.095913,-0.117876,-0.038183,-0.776491
tiger_global_management_llc,0.564186,0.666718,0.274038,-0.698932
berkshire_hathaway_inc,0.528679,0.701791,0.209938,-0.610553
s&p_500,0.714305,0.894260,0.486772,NaN


## Modigliani Ratio  
---

In [43]:
# Calculate the Modigliani Ratio
benchmark_annl_volatility = daily_returns['s&p_500'].std() * np.sqrt(252)
risk_ratios_df['modigliani_ratio'] = risk_ratios_df.iloc[:-1, 0].mul(
    benchmark_annl_volatility,
    axis='index'
) + 0  # 0 represents the risk-free rate of return
risk_ratios_df

,sharpe_ratio,sortino_ratio,calmar_ratio,information_ratio,modigliani_ratio
soros_fund_management_llc,-0.218074,-0.246034,-0.056719,-0.792278,-0.039983
paulson_&_co.inc.,-0.095913,-0.117876,-0.038183,-0.776491,-0.017585
tiger_global_management_llc,0.564186,0.666718,0.274038,-0.698932,0.103441
berkshire_hathaway_inc,0.528679,0.701791,0.209938,-0.610553,0.096931
s&p_500,0.714305,0.894260,0.486772,NaN,NaN
